In [0]:
%sql
CREATE CATALOG IF NOT EXISTS medalhao

In [0]:
%sql 
USE CATALOG medalhao;
CREATE SCHEMA IF NOT EXISTS bronze

In [0]:
%sql 
USE CATALOG medalhao;
CREATE SCHEMA IF NOT EXISTS silver

In [0]:
%sql 
USE CATALOG medalhao;
CREATE SCHEMA IF NOT EXISTS gold

In [0]:
catalogo = "medalhao"
bronze_db_name = "bronze"

In [0]:
from pyspark.sql import functions as F

def ingest_csv(nome_arquivo, nome_tabela):
   
    try:
        table_name = nome_tabela
        landing_path = f"/Volumes/medalhao/default/landing/{nome_arquivo}"

        # Leitura do arquivo CSV
        df = spark.read.csv(landing_path, header=True, inferSchema=True)

        # Validação: arquivo vazio
        if df.count() == 0:
            raise ValueError(f"O arquivo {nome_arquivo} está vazio ou não pôde ser lido.")

        # Adiciona timestamp de ingestão
        df_with_metadata = df.withColumn("ingestion_timestamp", F.current_timestamp())

        # Escrita no formato Delta
        df_with_metadata.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{bronze_db_name}.{table_name}")

        print(f"✅ Tabela bronze.{nome_tabela} criada com sucesso!\n")

    except Exception as e:
        print(f"Erro ao processar {nome_tabela}: {str(e)}")

In [0]:
ingest_csv("olist_customers_dataset.csv", "ft_consumidores")
ingest_csv("olist_geolocation_dataset.csv", "ft_geolocalizacao")
ingest_csv("olist_order_items_dataset.csv", "ft_itens_pedidos")
ingest_csv("olist_order_payments_dataset.csv", "ft_pagamentos_pedidos")
ingest_csv("olist_order_reviews_dataset.csv", "ft_avaliacoes_pedidos")
ingest_csv("olist_orders_dataset.csv", "ft_pedidos")
ingest_csv("olist_products_dataset.csv", "ft_produtos")
ingest_csv("olist_sellers_dataset.csv", "ft_vendedores")
ingest_csv("product_category_name_translation.csv", "dm_categoria_produtos_traducao")

In [0]:
dbutils.widgets.text("data_inicio_formatada", "Digite a data_inicio_formatada:", "Data I")
dbutils.widgets.text("data_fim_formatada", "Digite a data_fim_formatada:", "Data II")

In [0]:


import requests
import json

data_inicio_formatada = dbutils.widgets.get("data_inicio_formatada")
data_fim_formatada = dbutils.widgets.get("data_fim_formatada")

url = (
    f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo"
    f"(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)"
    f"?@dataInicial='{data_inicio_formatada}'"
    f"&@dataFinalCotacao='{data_fim_formatada}'"
    f"&$select=dataHoraCotacao,cotacaoCompra&$format=json"
)

headers = {
    'Cookie': 'BIGipServer~was_p_as3~was_p~pool_was_443_p=4275048876.47873.0000; JSESSIONID=0000HtHVCLHTsK-EWG0R60uBL2U:1dof89mke'
}

response = requests.get(url, headers=headers)
data = response.json()["value"]

df_cotacao_dolar = (spark.createDataFrame(data)
                    .withColumn("ingestion_timestamp", F.current_timestamp())
                    .withColumn("origem_do_dado", F.lit("API")))
display(df_cotacao_dolar)

df_cotacao_dolar.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{bronze_db_name}.dm_cotacao_dolar")